In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import optim, nn
import torch.nn.functional as F 

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import TensorDataset, DataLoader, random_split

In [ ]:
torch.manual_seed(0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "drive/MyDrive/Colab Notebooks/Datasets/Leather Defect Classification"
train_test_split = 0.2
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32,32)),
    ])
dataset = ImageFolder(root=path, transform=transform)
size = len(dataset)
test_size = int(train_test_split * size)
train_size = size - test_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
batch_size = 40
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class LeatherDefectCNN(nn.Module):
    def __init__(self, output_dim):
        super(LeatherDefectCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, output_dim)

        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
       
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = F.softmax(self.fc3(x),dim=1)
        return x              

In [ ]:
def calc_accuracy(a, y):                            
    a = torch.max(a, dim=1)[1]
    assert not torch.any(torch.isnan(a))
    return torch.mean((a == y).float()).item()

In [ ]:
model = LeatherDefectCNN(6)

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 100
loss_history = []
accuracy_history = []

for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for image, label in train_loader:
        z = model(image)
        loss = F.cross_entropy(z, label)
        accuracy = calc_accuracy(z, label)

        optimizer.zero_grad()      
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() / len(train_dataset) * batch_size
        epoch_accuracy += accuracy / len(train_dataset) * batch_size
    
    loss_history.append(epoch_loss)
    accuracy_history.append(epoch_accuracy)

    if epoch % 5 == 0:
        print("Epoch: {}, Loss: {}, Accuracy: {}".format(epoch, round(epoch_loss, 3), round(100*epoch_accuracy, 3)))

Epoch: 0, Loss: 1.44, Accuracy: 60.451
Epoch: 5, Loss: 1.431, Accuracy: 61.319
Epoch: 10, Loss: 1.427, Accuracy: 61.91
Epoch: 15, Loss: 1.439, Accuracy: 60.313
Epoch: 20, Loss: 1.429, Accuracy: 61.389
Epoch: 25, Loss: 1.428, Accuracy: 61.667
Epoch: 30, Loss: 1.413, Accuracy: 63.021
Epoch: 35, Loss: 1.389, Accuracy: 65.833
Epoch: 40, Loss: 1.428, Accuracy: 61.458
Epoch: 45, Loss: 1.379, Accuracy: 66.493
Epoch: 50, Loss: 1.383, Accuracy: 66.007
Epoch: 55, Loss: 1.377, Accuracy: 66.736
Epoch: 60, Loss: 1.373, Accuracy: 67.083
Epoch: 65, Loss: 1.379, Accuracy: 66.389
Epoch: 70, Loss: 1.373, Accuracy: 67.014
Epoch: 75, Loss: 1.37, Accuracy: 67.396
Epoch: 80, Loss: 1.356, Accuracy: 68.924
Epoch: 85, Loss: 1.417, Accuracy: 62.569
Epoch: 90, Loss: 1.412, Accuracy: 63.021
Epoch: 95, Loss: 1.413, Accuracy: 63.021


In [ ]:
num_epochs = 5
avg_loss = 0
avg_accuracy = 0

for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for image, label in test_loader:
        z = model(image)
        loss = F.cross_entropy(z, label)
        accuracy = calc_accuracy(z, label)

        epoch_loss += loss.item() / len(test_dataset) * batch_size
        epoch_accuracy += accuracy / len(test_dataset) * batch_size

    avg_loss += epoch_loss / num_epochs
    avg_accuracy += epoch_accuracy / num_epochs

print("Number of Epochs tested: {}, Average loss: {}, Average Accuracy: {}".format(num_epochs, round(avg_loss, 3), round(100*avg_accuracy, 3)))

Number of Epochs tested: 5, Average loss: 1.343, Average Accuracy: 70.056
